The goal of this notebook is to show how to subtitute analysed fields from one file to another file.      
This would occur typically if an analysis is performed for some time periods and have to replace other fields computed before.

__Method:__              
1. Open the new file and loop over the time
2. Find the corresponding time period in the old file
3. Replace all the fields that depend on time by the new one.

__Remark:__           
A backup copy of the old file is more than recommended!

In [1]:
using DIVAnd
using NCDatasets

In [2]:
datadir = "/home/ctroupin/"
olddatafile = joinpath(datadir, "Water_body_salinity_Baltic_monthly_decades_1955-2018_epsilon2_0.2_old.nc")
newdatafile = joinpath(datadir, "Water_body_salinity_Baltic_monthly_decades_1955-2018_epsilon2_0.2_new.nc")
@info(isfile(olddatafile) & isfile(newdatafile));

┌ Info: true
└ @ Main In[2]:4


In [3]:
function replace_field(oldfile::String, newfile::String)

    @info("Opening new file in *read* mode")
    NCDatasets.Dataset(newfile, "r") do ncnew

        @info("Opening old file in *edit* mode")
        NCDatasets.Dataset(oldfile, "a") do ncold

            @info("Read time variable and check for common periods")
            timesold = varbyattrib(ncold, standard_name="time")[1]
            timesnew = varbyattrib(ncnew, standard_name="time")[1]
            indexlist = [findall(t .== timesold)[1] for t in timesnew]
            @info("Common period numbers: $(indexlist)")
            @info("Common periods:\n $(timesold[indexlist])")
            @info("Common periods:\n $(timesnew[:])")
            @info("There are $(length(indexlist)) periods common to the 2 files")

            # Replace fields
            @info("Loop on the variables")
            for varname in keys(ncold)

                # Check if time is a dimension
                if "time" ∈ dimnames(ncold[varname])
                    @info("Variable '$(varname)' contains 'time' dimension")
                    varold = ncold[varname]
                    varnew = ncnew[varname]

                    ndims1 = length(size(varnew))
                    ndims2 = length(size(varold))

                    if ndims1 !== ndims2
                        @error("Variable $(varname) has a different dimension in the 2 files")
                    end

                    @debug("Number of dimensions: $(ndims)")

                    @info("Replacing values for variable: $(varname)")
                    if ndims1 == 1
                        varold[indexlist] = varnew[:]
                    elseif ndims1 == 2
                        varold[:,indexlist] = varnew[:,:]
                    elseif ndims1 == 3
                        varold[:,:,indexlist] = varnew[:,:,:]
                    elseif ndims1 == 4
                        varold[:,:,:,indexlist] = varnew[:,:,:,:]
                    else
                        @warn("Number of dimensions larger than 4")
                    end
                end
            end

        end
    end
end

replace_field (generic function with 1 method)

In [4]:
@time replace_field(olddatafile, newdatafile)

┌ Info: Opening new file in *read* mode
└ @ Main In[3]:3
┌ Info: Opening old file in *edit* mode
└ @ Main In[3]:6
┌ Info: Read time variable and check for common periods
└ @ Main In[3]:9
┌ Info: Common period numbers: [6, 7, 8, 9]
└ @ Main In[3]:13
┌ Info: Common periods:
│  [Dates.DateTime("1959-06-16T00:00:00"), Dates.DateTime("1959-07-16T00:00:00"), Dates.DateTime("1959-08-16T00:00:00"), Dates.DateTime("1959-09-16T00:00:00")]
└ @ Main In[3]:14
┌ Info: Common periods:
│  [Dates.DateTime("1959-06-16T00:00:00"), Dates.DateTime("1959-07-16T00:00:00"), Dates.DateTime("1959-08-16T00:00:00"), Dates.DateTime("1959-09-16T00:00:00")]
└ @ Main In[3]:15
┌ Info: There are 4 periods common to the 2 files
└ @ Main In[3]:16
┌ Info: Loop on the variables
└ @ Main In[3]:19
┌ Info: Variable 'time' contains 'time' dimension
└ @ Main In[3]:24
┌ Info: Replacing values for variable: time
└ @ Main In[3]:37
┌ Info: Variable 'climatology_bounds' contains 'time' dimension
└ @ Main In[3]:24
┌ Info: Replacing v

1213.894599 seconds (692.26 M allocations: 34.609 GiB, 0.31% gc time)
